In [1]:
# Core
import numpy as np
import pandas as pd
import os
import json

In [2]:
# Loads all fold metrics and puts them in a dataframe
def load_metrics(directory):
    # Load files
    filenames = os.listdir(directory)

    metrics = []
    for file in filenames:
        f = open(f'{directory}/{file}')
        
        metrics += [json.load(f)]

        f.close()

    fold_metrics = pd.DataFrame()
    for i, cur in enumerate(metrics):
        temp = {k:[v] for k,v in cur.items()}
        fold_metrics = pd.concat([fold_metrics, pd.DataFrame.from_dict(temp)], ignore_index=True)

    return fold_metrics

In [41]:
def load_preds(directory):
    # Load files
    f = open(f'{directory}/test_predictions.json')
    preds = [json.load(f)]
    f.close()

    return np.array(preds[0]['predictions'])

In [20]:
def load_csv(dataset, use_stopwords):
    if dataset not in ["train", "validate", "test"]:
        raise Exception("Invalid split.")
    if type(use_stopwords) != bool:
        raise Exception("Stop words must be specified in boolean.")
    
    stopwords = "With Stopwords"
    if not use_stopwords:
        stopwords = "Without Stopwords"
    df = pd.read_csv(f"./Data/{stopwords}/{dataset}.csv")
    
    return df

In [42]:
predictions = load_preds("./Models/hierarchical-initial/main")
len(predictions)

227

In [43]:
predictions = load_preds("./Models/hierarchical-initial/main")
# Create filtered sub model test dataset from predictions of main model
NA_indices = np.argwhere(predictions == 0).flatten()
A_indices = [x for x in range(len(predictions)) if x not in NA_indices]
len(A_indices)

156

In [44]:
predictions = load_preds("./Models/hierarchical-initial/main")
df = load_csv("test", True)

# Create filtered sub model test dataset from predictions of main model
NA_indices = np.argwhere(predictions == 0).flatten()
A_indices = [x for x in range(len(predictions)) if x not in NA_indices]
NA_truth_indices = df[df["Class"] == 0].index.tolist()
A_indices = [x for x in A_indices if x not in NA_truth_indices]

len(A_indices)

135

In [23]:
df = load_csv("test", True)
df = df[df["Class"] != 0]

print("Total:", len(df)) 
print("Explicit: ", len(df[df["Class"] == 1]))
print("Implicit: ", len(df[df["Class"] == 2]))

Total: 151
Explicit:  76
Implicit:  75


In [47]:
predictions = load_preds("./Models/hierarchical-initial/main")
df = load_csv("test", True)

# Create filtered sub model test dataset from predictions of main model
NA_indices = df[df["Class"] == 0].index.tolist()
A_indices = np.argwhere(predictions != 0).flatten()
A_indices = [x for x in A_indices if x not in NA_indices]

df = df.iloc[A_indices]
print("Total:", len(df)) 
print("Explicit: ", len(df[df["Class"] == 1]))
print("Implicit: ", len(df[df["Class"] == 2]))

Total: 135
Explicit:  73
Implicit:  62


In [9]:
preds = np.array(load_preds("./Models/hierarchical-initial/main"))
preds = np.where(preds == 1)[0]

print("Total:", len(preds)) 
print("Explicit: ", len(preds) - np.count_nonzero(preds == 1))
print("Implicit: ", np.count_nonzero(preds == 1))

Total: 156
Explicit:  155
Implicit:  1


In [7]:
preds = np.array(load_preds("./Models/hierarchical-final/main"))

print("Total:", len(preds)) 
print("Explicit: ", len(preds) - np.count_nonzero(preds == 1))
print("Implicit: ", np.count_nonzero(preds == 1))

Total: 227
Explicit:  77
Implicit:  150


In [7]:
dir = "./Models/experiment-with-stopwords/metrics"
metrics = load_metrics(dir)
metrics.mean()

epoch                       3.000000
eval_accuracy               0.718054
eval_f1                     0.715056
eval_loss                   1.025074
eval_precision              0.718842
eval_recall                 0.718054
eval_runtime                7.242840
eval_samples_per_second    29.298200
eval_steps_per_second       3.741400
dtype: float64

# Explore Incorrect Labels

In [8]:
# Default 50
pd.options.display.max_colwidth = 0

In [55]:
df = load_csv("test", True)
preds = np.array(load_preds("./Models/standard-final"))
preds[183]

2

## Multiclass

### Incorrect Non-Abusive

In [9]:
preds = np.array(load_preds("./Models/standard-final"))
df = load_csv("test", True)
df = df.iloc[preds == 0]
df[df["Class"] != 0].drop(df.columns[-10:], axis=1)

,Text,Class
6,mag CANDIDATE ako ayaw ko na ng patayan nung si RELATED nakaupo palagi na lang may patayan,2
24,hahahaha tawang tawa ako kay simp CANDIDATE lol HASHTAG RELATED CANDIDATE RELATED HASHTAG,1
64,sabi ng dormmate ko pag nanalo daw si CANDIDATE sa may 9 lilipad daw siya sa iabng bansa kasi wala na daw pag asa pinas sa isip isip ko edi lumipad ka di ka naman kawalan friend chariz syempre di ko sinabi baka umiyak e hahahahaha,2
99,wala munang kakausap sa akin today kahit mga one month lalo na mga CANDIDATE supporters :facevomiting:,2
105,half tropa half trapo PARTY CANDIDATE RELATED CANDIDATE RELATED LINK,2
128,modern day hudas aabangan ko aftr one year me posisyon na ito sa dilg o anumang ahensiya under CANDIDATE presidency LINK,2
169,magiingat po kayong lahat mga SUPPORTER s gutted i could not join todays mda super delikado daw talaga ng 1st trimester napagalitan na ako nung birthday rally ni mam CANDIDATE huhu praying for all of your safety god bless HASHTAG CANDIDATE SUPPORTER HASHTAG,2
172,MENTION binoto natin si vp CANDIDATE para sa mga nasa laylayan pero mga nasa laylayan binoto si :facewithspiraleyes: ay ambo,1
188,mareng CANDIDATE di ako makatulog kasalanan ito ni robin,2
213,MENTION ano sa tagalog ang thief edi CANDIDATE,2


### Incorrect Explicit

In [10]:
preds = np.array(load_preds("./Models/standard-final"))
df = load_csv("test", True)
df = df.iloc[preds == 1]
df[df["Class"] != 1].drop(df.columns[-10:], axis=1)

,Text,Class
3,ang gagaling mangakusa hahaha kala mo talaga naranasan ang CANDIDATE administration before,2
4,infer naman sa blogger at content creator friends ko 95 CANDIDATE RELATED may bilang na bilang lang na mga ampaw na suportado ang magnanakaw :facewithtearsofjoy: :cherryblossom:,2
14,tangina di talaga kapani paniwala na mas madami pa bumoto kay robin kesa kay CANDIDATE,2
16,MENTION CANDIDATE RELATED isa pa tong mapagpatol hahaha kayo talaga nagpapatrending kay SUPPORTER eh thank you sa pag campaign ha,2
17,may mga balita pa nga na malaki ang perang dinistribute sa local exec candidates mayors govs para masecure ang boto ng PARTY sa localities ang daming local politicians na tinalikuran ang prinsipyo para iendorso ang pangalan na CANDIDATE kahit poisoned na yan dati,2
18,maysakit na elderly simbahan anak ni binubully ang ina ang mga yan ang ginagawang props ng CANDIDATE RELATED campaign LINK,2
35,MENTION karen utang na loob pakitigil ang pagsasabing frontrunner si CANDIDATE nakakaumay mind conditioning,2
48,yung di mo kaya magalit sa mga magulang mo kahit ayaw nila kay CANDIDATE dahil ping CANDIDATE supporter naman sila :smilingfacewithtear:,0
70,si CANDIDATE at mga aquino ay mga anay na nagaaninyong mga tao ginagapang at sinisira nila ang mga isipan ng mga kabataan susunugin kayo sa impyerno sa dami ninyong kasinungalingan,2
75,to be honest nakakahiya yung mga pro CANDIDATE pero protrapo sa local election mahiya kayo sa mga laman nininyo,2


### Incorrect Implicit

In [11]:
preds = np.array(load_preds("./Models/standard-final"))
df = load_csv("test", True)
df = df.iloc[preds == 2]
df[df["Class"] != 2].drop(df.columns[-10:], axis=1)

,Text,Class
8,i meant tinakpan niya ng CANDIDATE RELATED sticker yung nakakabit na SUPPORTER sticker sa helmet niya haha twitter where is the edit button lol,0
19,MENTION mananahimik kan na kaya pag talo parin si CANDIDATE hahahaha,0
22,MENTION base sa mga narinig ko na mga botante na medyo may edad ayaw nila sa mga CANDIDATE dahil sa mga ginawa nila noon described as yawa or evil ang mga CANDIDATE malaking impact din ang mga active volunteers sa campaign rallies h2h and pagspread ng facts,0
37,hndi pa naman kasi nila alam ang ngyare sa panahon ng mga CANDIDATE kaya paniniwalaan nila yung mga kasinungalingan sa mga librong nabasa hahahaha LINK,0
38,dami kong naririnig na last minute change from vvm to not vp CANDIDATE another candidate tapos kingina super plot twist kapag iba nanalo magdadabog talaga ako :rollingonthefloorlaughing:,1
39,ay sus MENTION tinatanong pa ba yan dapat alam na ng kabila yan matik na dapat na kay presidrnt CANDIDATE at vp RELATED na yan SUPPORTER RELATED PARTY SUPPORTER RELATED SUPPORTER RELATED h2022 HASHTAG pula ang pilipinas LINK,0
42,MENTION haha make reading a habit basta CANDIDATE utak lamok,1
46,sorry hindi pa din talaga ko maka move on at sobrang sakit pa din talaga and my question again is bakit nga po ulit sinayang si vp CANDIDATE,0
51,galit na galit kayo pero patuloy ninyo naman pinakakinabangan mga pagawa ni CANDIDATE hays,0
52,SUPPORTER supporters on SUPPORTER s: okay lang hindi welleducated basta wellmannered hindi lang dapat matalino pero dapat makatao ka din also them: floods haha reacts on SUPPORTER s posts though nothing is laughable p SUPPORTER is for PARTY elections over LINK,0


## Hierarchical

### Incorrect Non-Abusive

In [12]:
preds = np.array(load_preds("./Models/hierarchical-final/main"))
df = load_csv("test", True)
df = df.iloc[preds == 0]
df[df["Class"] != 0].drop(df.columns[-10:], axis=1)

,Text,Class
24,hahahaha tawang tawa ako kay simp CANDIDATE lol HASHTAG RELATED CANDIDATE RELATED HASHTAG,1
27,oh pagod ka na ba kaka test inom ka muna hahahahahaha krazy LINK,2
28,MENTION sana kasi nagpapako si CANDIDATE sa krus baka nanalo sya,2
34,MENTION kelan kaya tayo makakaalis sa bulok na sistema na ito,1
64,sabi ng dormmate ko pag nanalo daw si CANDIDATE sa may 9 lilipad daw siya sa iabng bansa kasi wala na daw pag asa pinas sa isip isip ko edi lumipad ka di ka naman kawalan friend chariz syempre di ko sinabi baka umiyak e hahahahaha,2
99,wala munang kakausap sa akin today kahit mga one month lalo na mga CANDIDATE supporters :facevomiting:,2
105,half tropa half trapo PARTY CANDIDATE RELATED CANDIDATE RELATED LINK,2
120,SUPPORTER po pala talaga ako just got home from SUPPORTER victory party in solaire exclusive sa mga alta like me pag inggit pikit ok tiyaga na lang kayo dyan sa party ninyo sa ateneo na walang pashanghai SUPPORTER RELATED PARTY SUPPORTER CANDIDATE HASHTAG LINK,2
128,modern day hudas aabangan ko aftr one year me posisyon na ito sa dilg o anumang ahensiya under CANDIDATE presidency LINK,2
136,MENTION ang perfect naman s pagiging toxic ng tao to bravo god bless,1


### Incorrect Abusive

In [13]:
preds = np.array(load_preds("./Models/hierarchical-final/main"))
df = load_csv("test", True)
df = df.iloc[preds == 1]
df[df["Class"] == 0].drop(df.columns[-10:], axis=1)

,Text,Class
8,i meant tinakpan niya ng CANDIDATE RELATED sticker yung nakakabit na SUPPORTER sticker sa helmet niya haha twitter where is the edit button lol,0
19,MENTION mananahimik kan na kaya pag talo parin si CANDIDATE hahahaha,0
22,MENTION base sa mga narinig ko na mga botante na medyo may edad ayaw nila sa mga CANDIDATE dahil sa mga ginawa nila noon described as yawa or evil ang mga CANDIDATE malaking impact din ang mga active volunteers sa campaign rallies h2h and pagspread ng facts,0
37,hndi pa naman kasi nila alam ang ngyare sa panahon ng mga CANDIDATE kaya paniniwalaan nila yung mga kasinungalingan sa mga librong nabasa hahahaha LINK,0
39,ay sus MENTION tinatanong pa ba yan dapat alam na ng kabila yan matik na dapat na kay presidrnt CANDIDATE at vp RELATED na yan SUPPORTER RELATED PARTY SUPPORTER RELATED SUPPORTER RELATED h2022 HASHTAG pula ang pilipinas LINK,0
48,yung di mo kaya magalit sa mga magulang mo kahit ayaw nila kay CANDIDATE dahil ping CANDIDATE supporter naman sila :smilingfacewithtear:,0
51,galit na galit kayo pero patuloy ninyo naman pinakakinabangan mga pagawa ni CANDIDATE hays,0
52,SUPPORTER supporters on SUPPORTER s: okay lang hindi welleducated basta wellmannered hindi lang dapat matalino pero dapat makatao ka din also them: floods haha reacts on SUPPORTER s posts though nothing is laughable p SUPPORTER is for PARTY elections over LINK,0
58,ayaw nininyo ba ng may clear platforms that is already gaining CANDIDATE or mas gusto nininyo CANDIDATE tas wala pa ding platforms:meltingface: :facewithpeekingeye:,0
67,halatang may magic talaga e napapaisip tuloy ako kung ano talaga result kung walang pandarayang nagaganap kung ilan ang lamang ni vp kay CANDIDATE,0


### Incorrect Explicit

In [14]:
preds = np.array(load_preds("./Models/hierarchical-final/main"))
df = load_csv("test", True)
df = df.iloc[preds == 1]
df.reset_index(inplace=True)

preds = np.array(load_preds("./Models/hierarchical-final/sub"))
df = df.iloc[preds == 0]
df[df["Class"] != 1].drop(df.columns[-10:], axis=1)

,index,Text,Class
1,1,katapat ng pila ko SUPPORTER supporters ng mga marites sabi ng isa na malinis si CANDIDATE mga CANDIDATE kasi magalin manira ng iba dati pa lang mangdaraya may vote buying pa sabay tawa sila ng sabay sabay ang tatanda nila at alam na nila pinag sasabi nila :thinkingface: :unamusedface:,2
3,3,ang gagaling mangakusa hahaha kala mo talaga naranasan ang CANDIDATE administration before,2
4,4,infer naman sa blogger at content creator friends ko 95 CANDIDATE RELATED may bilang na bilang lang na mga ampaw na suportado ang magnanakaw :facewithtearsofjoy: :cherryblossom:,2
8,10,mas kawawa daw pilipinas kapag si CANDIDATE ang maupo lah hahaha mas kawawa ka kamo,2
12,14,tangina di talaga kapani paniwala na mas madami pa bumoto kay robin kesa kay CANDIDATE,2
14,16,MENTION CANDIDATE RELATED isa pa tong mapagpatol hahaha kayo talaga nagpapatrending kay SUPPORTER eh thank you sa pag campaign ha,2
16,18,maysakit na elderly simbahan anak ni binubully ang ina ang mga yan ang ginagawang props ng CANDIDATE RELATED campaign LINK,2
25,35,MENTION karen utang na loob pakitigil ang pagsasabing frontrunner si CANDIDATE nakakaumay mind conditioning,2
38,52,SUPPORTER supporters on SUPPORTER s: okay lang hindi welleducated basta wellmannered hindi lang dapat matalino pero dapat makatao ka din also them: floods haha reacts on SUPPORTER s posts though nothing is laughable p SUPPORTER is for PARTY elections over LINK,0
50,70,si CANDIDATE at mga aquino ay mga anay na nagaaninyong mga tao ginagapang at sinisira nila ang mga isipan ng mga kabataan susunugin kayo sa impyerno sa dami ninyong kasinungalingan,2


### Incorrect Implicit

In [15]:
preds = np.array(load_preds("./Models/hierarchical-final/main"))
df = load_csv("test", True)
df = df.iloc[preds == 1]
df.reset_index(inplace=True)

preds = np.array(load_preds("./Models/hierarchical-final/sub"))
df = df.iloc[preds == 1]
df[df["Class"] != 2].drop(df.columns[-10:], axis=1)

,index,Text,Class
6,8,i meant tinakpan niya ng CANDIDATE RELATED sticker yung nakakabit na SUPPORTER sticker sa helmet niya haha twitter where is the edit button lol,0
17,19,MENTION mananahimik kan na kaya pag talo parin si CANDIDATE hahahaha,0
19,22,MENTION base sa mga narinig ko na mga botante na medyo may edad ayaw nila sa mga CANDIDATE dahil sa mga ginawa nila noon described as yawa or evil ang mga CANDIDATE malaking impact din ang mga active volunteers sa campaign rallies h2h and pagspread ng facts,0
26,37,hndi pa naman kasi nila alam ang ngyare sa panahon ng mga CANDIDATE kaya paniniwalaan nila yung mga kasinungalingan sa mga librong nabasa hahahaha LINK,0
27,38,dami kong naririnig na last minute change from vvm to not vp CANDIDATE another candidate tapos kingina super plot twist kapag iba nanalo magdadabog talaga ako :rollingonthefloorlaughing:,1
28,39,ay sus MENTION tinatanong pa ba yan dapat alam na ng kabila yan matik na dapat na kay presidrnt CANDIDATE at vp RELATED na yan SUPPORTER RELATED PARTY SUPPORTER RELATED SUPPORTER RELATED h2022 HASHTAG pula ang pilipinas LINK,0
29,41,MENTION CANDIDATE wag na di na need ipasa legit ang kabobohang taglay mo promise,1
34,48,yung di mo kaya magalit sa mga magulang mo kahit ayaw nila kay CANDIDATE dahil ping CANDIDATE supporter naman sila :smilingfacewithtear:,0
37,51,galit na galit kayo pero patuloy ninyo naman pinakakinabangan mga pagawa ni CANDIDATE hays,0
42,58,ayaw nininyo ba ng may clear platforms that is already gaining CANDIDATE or mas gusto nininyo CANDIDATE tas wala pa ding platforms:meltingface: :facewithpeekingeye:,0
